# BULID COMBINED MODEL

In [45]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Conv2D, Dense, Flatten
from keras.layers import GlobalAveragePooling2D, Reshape
from tensorflow.keras.applications import EfficientNetB3, MobileNetV2
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input


In [46]:
def build_combined_model():
    input_shape = (224, 224, 3)
    input_layer = Input(shape=input_shape)

    efficientnet_model = EfficientNetB3(include_top=False, input_tensor=input_layer, weights='imagenet')
    mobilenet_model = MobileNetV2(include_top=False, input_tensor=input_layer, weights='imagenet')

    efficientnet_layers = efficientnet_model.get_layer('block1a_project_bn').output
    mobilenet_layers = mobilenet_model.get_layer('expanded_conv_depthwise_BN').output

    # Menambahkan Conv2D layer sebelum concatenate
    previous_layer = Conv2D(56, kernel_size=(2,2), strides=(2,2), activation='relu', padding='same')(input_layer)

    
    print("Previous Layer Shape:", previous_layer.shape)
    print("EfficientNet Layers Shape:", efficientnet_layers.shape)
    print("MobileNet Layers Shape:", mobilenet_layers.shape)

    # Concatenate
    merged_layers = Concatenate(axis=-1)([previous_layer, efficientnet_layers, mobilenet_layers])

    print("Merged Layers Shape:", merged_layers.shape)

    flattened_layer = Flatten()(merged_layers)
    dense_layer = Dense(128, activation='relu')(flattened_layer)
    output_layer = Dense(4, activation='softmax')(dense_layer)
    print("Output Layer Shape:", output_layer.shape)
    
    combined_model = Model(inputs=input_layer, outputs=output_layer)

    return combined_model


In [47]:
combined_model = build_combined_model()
# combined_model.summary()

Previous Layer Shape: (None, 112, 112, 56)
EfficientNet Layers Shape: (None, 112, 112, 24)
MobileNet Layers Shape: (None, 112, 112, 32)
Merged Layers Shape: (None, 112, 112, 112)
Output Layer Shape: (None, 4)


In [48]:
combined_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_2 (Rescaling)     (None, 224, 224, 3)          0         ['input_2[0][0]']             
                                                                                                  
 normalization_1 (Normaliza  (None, 224, 224, 3)          7         ['rescaling_2[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_3 (Rescaling)     (None, 224, 224, 3)          0         ['normalization_1[0][0]'

In [49]:
optimizer = Adam(learning_rate=0.0001)  

combined_model.compile(optimizer = optimizer,loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [50]:
combined_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# PEMBAGIAN TRAIN TEST VAL

In [51]:
import os
import shutil
from sklearn.model_selection import train_test_split
dataset_path = 'C:/SEMESTER 5/STECHOQ/PROJECT/data/brain/Testing'
output_path = 'Data Combine'

In [52]:
train_path = os.path.join(output_path, 'train')
val_path = os.path.join(output_path, 'val')
test_path = os.path.join(output_path, 'test')

os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

In [53]:
def copy_files(file_paths, destination_folder):
    for file_path in file_paths:
        shutil.copy(file_path, destination_folder)

# Assuming each subdirectory in 'dataset_path' corresponds to a class
classes = os.listdir(dataset_path)

In [54]:

for class_name in classes:
    class_path = os.path.join(dataset_path, class_name)
    class_images = [os.path.join(class_path, img) for img in os.listdir(class_path)]

    # Split the data into train, val, and test
    train_images, test_images = train_test_split(class_images, test_size=0.5, random_state=42)
    train_images, val_images = train_test_split(train_images, test_size=0.2, random_state=42)

    # Create subdirectories for each class in train, val, and test
    class_train_path = os.path.join(train_path, class_name)
    class_val_path = os.path.join(val_path, class_name)
    class_test_path = os.path.join(test_path, class_name)

    os.makedirs(class_train_path, exist_ok=True)
    os.makedirs(class_val_path, exist_ok=True)
    os.makedirs(class_test_path, exist_ok=True)

    # Copy images to the corresponding folders
    copy_files(train_images, class_train_path)
    copy_files(val_images, class_val_path)
    copy_files(test_images, class_test_path)

# Preprocessing

In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

training_dir = r"Data Combine/train"
validation_dir = r"Data Combine/val/"
test_dir = r"Data Combine/test"

categories = ["glioma_tumor","meningioma_tumor","no_tumor","pituitary_tumor"]

In [56]:
img_size = (224,224)

training_data = []

def create_training_data():
    for category in categories:
        path = os.path.join(training_dir,category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array,img_size) 
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
create_training_data()
print("Berhasil Create Training Data")

Berhasil Create Training Data


In [57]:
X_train = []
Y_train = []

for features,label in training_data:
    X_train.append(features)
    Y_train.append(label)
X_train = np.array(X_train).reshape(-1,224,224)
X_train = X_train.astype('float32')/255.0  
X_train = X_train.reshape(-1,224,224,3)
print(X_train.shape)

(2140, 224, 224, 3)


In [58]:
# IMG_SIZE = 150

validation_data = []

def create_validation_data():
    for category in categories:
        path = os.path.join(validation_dir,category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array,img_size) 
                validation_data.append([new_array,class_num])
            except Exception as e:
                pass
create_validation_data()
print("Berhasil Create Validation Data")

Berhasil Create Validation Data


In [59]:
X_val = []
Y_val = []
for features,label in validation_data:
    X_val.append(features)
    Y_val.append(label)
X_val = np.array(X_val).reshape(-1,224,224)
X_val = X_val.astype('float32')/255.0  
X_val = X_val.reshape(-1,224,224,3)
print(X_val.shape)

(921, 224, 224, 3)


In [60]:
test_data = []

def create_test_data():
    for category in categories:
        path = os.path.join(validation_dir,category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array,img_size) 
                validation_data.append([new_array,class_num])
            except Exception as e:
                pass
create_test_data()
print("Berhasil Create test Data")

Berhasil Create test Data


In [61]:
X_test = []
Y_test = []
for features,label in validation_data:
    X_test.append(features)
    Y_test.append(label)
X_test = np.array(X_test).reshape(-1,224,224)
X_test = X_test.astype('float32')/255.0  
X_test = X_test.reshape(-1,224,224,3)
print(X_test.shape)

(1842, 224, 224, 3)


In [62]:
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding
Y_train = to_categorical(Y_train, num_classes = 4)
Y_val = to_categorical(Y_val, num_classes = 4)
Y_test = to_categorical(Y_test, num_classes = 4)

print("x_train shape",X_train.shape)
print("x_val shape",X_val.shape)
print("x_test shape",X_test.shape)
print("y_train shape",Y_train.shape)
print("y_val shape",Y_val.shape)
print("y_test shape",Y_test.shape)

x_train shape (2140, 224, 224, 3)
x_val shape (921, 224, 224, 3)
x_test shape (1842, 224, 224, 3)
y_train shape (2140, 4)
y_val shape (921, 4)
y_test shape (1842, 4)


# TRAINING DATA

In [63]:

history = combined_model.fit(X_train,Y_train, batch_size=64, epochs = 12, validation_data = (X_val,Y_val))

Epoch 1/12
34/34 [==============================] - 126s 3s/step - loss: 621.8327 - accuracy: 0.5098 - val_loss: 730.6208 - val_accuracy: 0.3974
Epoch 2/12
34/34 [==============================] - 105s 3s/step - loss: 23.1589 - accuracy: 0.7491 - val_loss: 380.7246 - val_accuracy: 0.3974
Epoch 3/12
 9/34 [======>.......................] - ETA: 1:12 - loss: 11.8707 - accuracy: 0.8108

KeyboardInterrupt: 